In [190]:
from solid import *
from solid.utils import *
import pandas as pd
from subprocess import run

In [191]:
SHEET_ID = '1YMguWXBo50LdKzekR7vHyMYXFv2_KdIs1KA8RjC_I2Y'
SHEET_NAME = 'pallets_baybeeeee'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
pallets_list = pd.read_csv(url)
pallets_list


,pallet,layer,h,w,count_x,d,count_y
0,sally,1,1.50,100.0,1,7.2,9
1,sally,2,2.00,7.2,3,120.0,1
2,sally,3,8.75,7.2,3,7.2,3
3,sally,4,2.00,7.2,3,120.0,1
4,dean,1,2.50,120.0,1,9.5,7
5,dean,2,2.00,9.8,3,100.0,1
6,dean,3,7.80,10.0,3,9.5,3
7,dean,4,2.00,120.0,1,9.5,3
8,dean,5,1.50,8.5,3,100.0,1
9,charles,1,2.50,120.0,1,9.5,8


In [192]:
class Pallet:
    def __init__(self, width, depth):
        self.width = width
        self.depth = depth
        self.tallness = 0
        self.layers = []

    def add_layer(self, item_width, item_depth, item_height, x_count, y_count, xy_skip = []):

        x_offset = 0 if x_count < 2 else item_width + (self.width - (item_width * x_count)) / (x_count-1)

        y_offset = 0 if y_count < 2 else item_depth + (self.depth - (item_depth * y_count)) / (y_count-1)

        # TODO get rid of this instantiation, messes with xy_skip = (1,1). Need to be able to instantiate a solidpython object which is nothing.
        layer = cube([item_width, item_depth, item_height])

        layer = translate([0,0,self.tallness])(layer)
        for x in range(x_count):
            for y in range(y_count):
                if (x+1,y+1) not in xy_skip:
                    item = cube([item_width, item_depth, item_height])
                    item = translate([x * x_offset, y * y_offset, self.tallness])(item)
                    layer += item

        self.layers.append(layer)
        self.tallness += item_height

    def squish(self):
        out = union()(*self.layers)
        return out


In [193]:
# # Create a pallet with a width of 120 and a depth of 120
# pallet = Pallet(100, 120)


# # Add three layers to the pallet
# pallet.add_layer(item_width=7.2, item_depth=120, item_height=2, x_count=3, y_count=1)
# pallet.add_layer(item_width=7.2, item_depth=7.2, item_height=8.75, x_count=3, y_count=3)
# pallet.add_layer(item_width=7.2, item_depth=120, item_height=2, x_count=3, y_count=1)
# pallet.add_layer(item_width=100, item_depth=7.2, item_height=1.5, x_count=1, y_count=9)



# # Render the pallet to a file
# pallet.render(file_name="example_pallet.scad")

In [194]:
pallets_dict = {}

for name in pallets_list['pallet'].unique():
    pwidth = pallets_list.loc[pallets_list['pallet']==name,'w'].max()
    pdepth = pallets_list.loc[pallets_list['pallet']==name,'d'].max()
    pallets_dict[name] = Pallet(pwidth,pdepth)


for index, row in pallets_list.iterrows():
    pallets_dict[row['pallet']].add_layer(item_width=row['w'], item_depth=row['d'], item_height=row['h'], x_count=row['count_x'], y_count=row['count_y'])


for k, v in pallets_dict.items():
    stfn = k + ".scad"
    obj = v.squish()
    scad_render_to_file(obj, stfn)

